# Institute for Behavioral Genetics International Statistical Genetics 2023 Workshop 

## Advanced Hail Functionality

This notebook is a grab bag of more advanced Hail functionality.

### Approximate CDF

Normally computing quantiles or the median requires sorting an entire dataset. Hail uses a sophisticated data structure to get provably good approximations of all quantiles without sorting the data, providing buckets, or using unbounded memory.

In [ ]:
import os
# Give Hail a bunch of RAM. Must happen before using Hail.
os.environ['PYSPARK_SUBMIT_ARGS'] = '--executor-memory 16G --driver-memory 16G pyspark-shell'

In [ ]:
import hail as hl
hl.plot.output_notebook()

In [ ]:
mt = hl.read_matrix_table('resources/hgdp-tgp-rare-variants.mt')
mt = hl.variant_qc(mt)
call_rate_cdf = mt.aggregate_rows(hl.agg.approx_cdf(mt.variant_qc.call_rate))
call_rate_cdf

In [ ]:
cdf_plot = hl.plot.cdf(call_rate_cdf, title='Approximate CDF of Call Rate', legend='call rate')
hl.plot.show(cdf_plot)

In [ ]:
mt = hl.read_matrix_table('resources/hgdp-tgp-rare-variants.mt')
mt = hl.variant_qc(mt)
ref_af_cdf = mt.aggregate_rows(hl.agg.approx_cdf(mt.variant_qc.AF[0]))
cdf_plot = hl.plot.cdf(ref_af_cdf, title='Approximate CDF of Reference AF', legend='af')
hl.plot.show(cdf_plot)

You can also ask directly for the median:

In [ ]:
mt.aggregate_rows(hl.agg.approx_median(mt.variant_qc.AF[0]))

### PCA on Unusual Values

Flexible, general-purpose methods enable analysts to explore data sets with novel statistics.

In [ ]:
mt = hl.read_matrix_table('resources/hgdp-subset-3.mt')
mt = mt.filter_rows(hl.agg.any(hl.is_missing(mt.GT)))
mt = mt.annotate_entries(
    is_missing = hl.is_missing(mt.GT)
)
mt = mt.annotate_rows(
    is_missing_stats = hl.agg.stats(mt.is_missing)
)
mt = mt.annotate_entries(
    normed_is_missing = (mt.is_missing - mt.is_missing_stats.mean) / mt.is_missing_stats.stdev
)
_, scores, _ = hl.pca(mt.normed_is_missing, k=2)
hl.plot.show(hl.plot.scatter(scores.scores[0], scores.scores[1]))

### LD Prune

In [ ]:
?hl.ld_prune

In [ ]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')
print(f'Before pruning we have: {mt.count_rows()}')
pruned_variants = hl.ld_prune(mt.GT)
pruned_variants.write('output/pruned_variants.ht', overwrite=True)
pruned_variants = hl.read_table('output/pruned_variants.ht')

mt = mt.filter_rows(hl.is_defined(pruned_variants[mt.row_key]))
print(f'After pruning we have: {mt.count_rows()}')

### Kinship Estimators

Hail supports a number of different kinship estimators.

Getting PC Relate to produce good-looking results is tricky! Here we see what happens when you don't quality control the variants well enough.

In [ ]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')

pc_kin = hl.pc_relate(mt.GT, 0.1, k=4, statistics='kin20', min_kinship=0.1)
pc_kin.write('output/pc_kin.ht', overwrite=True)
pc_kin = hl.read_table('output/pc_kin.ht')

hl.plot.show(
    hl.plot.scatter(
        pc_kin.kin,
        pc_kin.ibd0,
        width=400,
        height=400,
        size=3
    )
)

In [ ]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')

king_kin = hl.king(mt.GT)
king_kin = king_kin.filter_entries(king_kin.phi > 0.1).entries()
king_kin.write('output/king_kin.ht', overwrite=True)
king_kin = hl.read_table('output/king_kin.ht')

hl.plot.show(
    hl.plot.histogram(
        king_kin.phi
    )
)

In [ ]:
king_kin.filter(king_kin.phi < 0.45).show()

Hail also supports identity-by-descent calculation but it's currently broken for the new Apple M1 chips because it uses some fast native code that hasn't been compiled for M1 yet. Expect a fix soon!

### Polygenic Score Calculation

In this section, I import a height polygenic score from the [PGS Catalog](https://www.pgscatalog.org/score/PGS000297/), and use it to calculate the polygenic score in our toy dataset. Our toy dataset does not have enough shared variants with the score to produce useful estimates, but the code below could be effectively applied to a larger, quality-controlled dataset.

In [ ]:
ht = hl.import_table('resources/height-polygenic-score.txt', comment='#', impute=True)
ht = ht.key_by(
    locus = hl.locus(hl.str(ht.chr_name), ht.chr_position)
)
ht.write('output/height-polygenic-score.ht', overwrite=True)

In [ ]:
ht = hl.read_table('output/height-polygenic-score.ht')

In [ ]:
ht.show()

In [ ]:
mt = hl.read_matrix_table('resources/1kg.mt')
mt = hl.variant_qc(mt)
mt = mt.annotate_rows(score=ht[mt.locus])

mt = mt.annotate_rows(is_flipped = (
    hl.case()
    .when(mt.score.effect_allele == mt.alleles[0], True)
    .when(mt.score.effect_allele == mt.alleles[1], False)
    .or_missing()
))
mt = mt.annotate_rows(
    mean_gt=2 * hl.if_else(mt.is_flipped, mt.variant_qc.AF[0], mt.variant_qc.AF[1])
)
mt = mt.annotate_entries(
    n_effect_alleles = hl.if_else(
        mt.is_flipped,
        2 - mt.GT.n_alt_alleles(),
        mt.GT.n_alt_alleles()
    )
)
mt = mt.annotate_cols(
    prs = hl.agg.sum(mt.score.effect_weight * hl.coalesce(mt.n_effect_alleles, mt.mean_gt)),
    n_useful_variants = hl.agg.sum(hl.is_defined(mt.score.effect_weight))
)
mt.cols().show()

### LD Score

Hail also has utilities for simulating phenotypes, calculating LD Scores, and running LD Score regression.

In [ ]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')
mt = hl.experimental.ldscsim.simulate_phenotypes(mt, mt.GT, h2=0.5)
mt.y.show()

In [ ]:
betas = hl.linear_regression_rows(y=mt.y, x=mt.GT.n_alt_alleles(), covariates=[1.0])

In [ ]:
betas.show()

In [ ]:
?hl.experimental.ld_score

In [ ]:
ht_scores = hl.experimental.ld_score(entry_expr=mt.GT.n_alt_alleles(),
                                     locus_expr=mt.locus,
                                     radius=1e6)


betas = betas.annotate(z_score = betas.beta / betas.standard_error)
betas = betas.annotate(chi_sq_statistic = betas.z_score ** 2)

ht = mt.rows()

ht_results = hl.experimental.ld_score_regression(
    weight_expr=ht_scores[ht.locus].univariate,
    ld_score_expr=ht_scores[ht.locus].univariate,
    chi_sq_exprs=betas[ht.key].chi_sq_statistic,
    n_samples_exprs=betas[ht.key].n
)

In [ ]:
ht_results.write('output/ldsr.ht', overwrite=True)
ldsr = hl.read_table('output/ldsr.ht')
ldsr.show()

### Annotation Database

The Hail team maintains a database of common variant annotations in Google Cloud Storage and S3. These commands will only work when executed inside a cluster with access to Google Cloud Storage or S3. They will not work on your laptop.

A full list of available annotations can be found [in the Hail docs](https://hail.is/docs/0.2/annotation_database_ui.html).

In [ ]:
mt = hl.read_matrix_table('resources/1kg.mt')

db = hl.experimental.DB(region='us', cloud='aws')
mt = db.annotate_rows_db(
    mt,
    'CADD', 'GTEx_eQTL_Adipose_Subcutaneous_all_snp_gene_associations', 'gnomad_ld_scores_afr'
)
mt.rows().show()

### VEP

Hail also supports VEP annotation. This requires a specially configured cluster. The code below will not work on your laptop.

In [ ]:
mt = hl.read_matrix_table('resources/1kg.mt')
mt = hl.vep(mt)
mt.vep.show()

In [ ]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')
mt.vep.show()

In [ ]:
mt = mt.annotate_rows(
    interesting_cnsq = mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains("stop_gained"))
)
mt = mt.filter_rows(hl.is_defined(mt.interesting_cnsq))
mt.interesting_cnsq.show(n=30)